# Complete IQ Signal Analysis for MPACT DroneRC RF Dataset (Parallel, High-Res Plots)

Tailored for MPACT DroneRC RF Dataset: Loads .mat IQ files from 17 drone RCs at 2.4 GHz, extracts features for model fingerprinting, generates/saves high-res plots (spectrograms, constellations, I/Q distributions with narrow bins), and trains multi-class ML. Uses 10 CPU cores; truncates to 1M samples.

Before running: Update `DATASET_DIR` to your MPACT folder.

In [ ]:
# Cell 1: Imports & Configuration (Updated for MPACT)
import os
import numpy as np
import pandas as pd
import scipy.io
from scipy import stats, signal
from scipy.signal import spectrogram, welch, hilbert, find_peaks
from scipy.fft import fftshift
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import multiprocessing
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report, accuracy_score
import joblib
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')
import gc
import logging  # NEW: Structured logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Progress widgets
from ipywidgets import IntProgress
from IPython.display import display
from threading import Thread
import time

# ---------- CONFIG (MPACT-Tailored) ----------
DATASET_DIR = r"/home/sujal-singh/Project/Sujal Singh/Data Extraction/MPACT_DroneRC_RF_Dataset (1)/MPACT_DroneRC_RF_Dataset"  # MPACT path
OUTPUT_DIR = "COMPLETE_IQ_ANALYSIS"
TEST_MODE = False
MAX_FILES_PER_DEVICE = 1 if TEST_MODE else None
TRUNCATE_LEN_TEST = 200000
MAX_SIGNAL_LEN = 1000000  # ~50µs at 20GS/s; suits 0.25ms bursts
HIST_BINS = 150  # NEW: Higher bins for narrower rectangles/higher res in I/Q dists
DPI_SAVE = 300
DPI_SAVE_DL = 400
DL_IMAGE_SIZE = (224, 224)
FIXED_CLIP_STD = 10.0
CLIP_OUTLIERS = True  # NEW: Toggle IQR clipping
NORMALIZE_BEFORE_FEATURES = True
NUM_CORES = min(2, multiprocessing.cpu_count())  # UPDATED: Use up to 10 cores

# MPACT defaults (confirmed from dataset)
DEFAULT_FS = 20e9
DEFAULT_FC = 2.4e9

# Ensure subdirs
subdirs = [
    'raw_iq_signals', 'magnitude_spectrogram_plots', 'phase_spectrogram_plots',
    'instantaneous_frequency_plots', 'original_constellation_plots', 'processed_constellation_plots', 'iq_time_series_plots',
    'extracted_features', 'trained_models', 'analysis_results', 'combined_visualizations',
    'dl_ready_magnitude_spectrograms', 'dl_ready_phase_spectrograms',
    'dl_ready_original_constellation_plots', 'dl_ready_processed_constellation_plots',
    'dl_ready_instantaneous_frequency_plots', 'dl_ready_iq_time_series_plots',
    'i_distribution_plots', 'q_distribution_plots', 'magnitude_phase_plots',
    'dl_ready_i_distribution_plots', 'dl_ready_q_distribution_plots', 'dl_ready_magnitude_phase_plots',
    'feature_importance'  # NEW: For ML viz
]
for sd in subdirs:
    os.makedirs(os.path.join(OUTPUT_DIR, sd), exist_ok=True)

print(f"DATASET_DIR: {DATASET_DIR}")
print(f"OUTPUT_DIR: {OUTPUT_DIR}")
print(f"TEST_MODE: {TEST_MODE}, MAX_FILES_PER_DEVICE: {MAX_FILES_PER_DEVICE}")
print(f"MAX_SIGNAL_LEN: {MAX_SIGNAL_LEN}")
print(f"HIST_BINS: {HIST_BINS}")
print(f"Using {NUM_CORES} CPU cores for parallel processing")

In [2]:
# Cell 2: Robust loader (Enhanced fs parsing for MPACT .mat)
def load_iq_signal(mat_path, test_mode=TEST_MODE, truncate_len=TRUNCATE_LEN_TEST, max_signal_len=MAX_SIGNAL_LEN):
    try:
        mat = scipy.io.loadmat(mat_path, squeeze_me=True, struct_as_record=False)
        # Find signal (MPACT: often 'data' or struct.Data)
        candidate = None
        if 'data' in mat:
            candidate = mat['data']
        else:
            for k, v in mat.items():
                if k.startswith('__'): continue
                if k.lower() in ('iq', 'signal', 'data'):
                    candidate = v
                    break
        if candidate is None:
            for k, v in mat.items():
                if not k.startswith('__') and isinstance(v, (np.ndarray, list)):
                    candidate = v
                    break

        if candidate is None:
            raise ValueError('No suitable signal found in MAT file')

        if hasattr(candidate, 'Data'):
            sig = np.squeeze(candidate.Data)
        elif isinstance(candidate, np.ndarray) and candidate.dtype.names and 'Data' in candidate.dtype.names:
            struct0 = candidate[0] if candidate.shape else candidate
            sig = np.squeeze(struct0['Data'])
        else:
            sig = np.squeeze(candidate)

        metadata = {
            'device': os.path.basename(os.path.dirname(mat_path)),  # e.g., 'DJI_Mini_2'
            'filename': os.path.basename(mat_path),
            'device_class': os.path.basename(os.path.dirname(mat_path)),  # UPDATED: Full device name for multi-class
            'fs': DEFAULT_FS,  # MPACT default
            'fc': DEFAULT_FC,
            'original_length': int(np.size(sig)),
            'is_complex': np.iscomplexobj(sig)
        }
        # Enhanced fs parsing (scan more fields)
        fs_candidates = ['XInc', 'Fs', 'sampling_rate', 'SampleRate', 'fs']
        for field in fs_candidates:
            try:
                if hasattr(candidate, field):
                    val = float(getattr(candidate, field))
                    if val > 0:
                        metadata['fs'] = 1.0 / val if field == 'XInc' else val
                        break
                elif field in mat:
                    val = float(mat[field])
                    if val > 0:
                        metadata['fs'] = val
                        break
            except:
                pass

        # Real to analytic (Hilbert for oscillo real captures)
        if not np.iscomplexobj(sig):
            real_sig = np.asarray(sig, dtype=np.float64)
            rms = np.sqrt(np.mean(real_sig**2) + 1e-12)
            if rms > 0:
                real_sig /= rms
            metadata['normalized_before_hilbert'] = True
            analytic = hilbert(real_sig)
            sig = analytic.astype(np.complex64)
            metadata['converted_to_complex'] = True
            metadata['is_complex'] = True
        else:
            sig = np.asarray(sig, dtype=np.complex64)

        # Truncate
        truncate_len = truncate_len if test_mode else max_signal_len
        if sig.size > truncate_len:
            sig = sig[:int(truncate_len)]
            metadata['truncated'] = True
            metadata['truncated_length'] = int(truncate_len)

        del mat, candidate
        gc.collect()
        return sig, metadata
    except Exception as e:
        logging.error(f"Load error {mat_path}: {e}")
        return None, None

In [3]:
# Cell 3: Prepare IQ (Toggle clipping)
def prepare_iq_for_plotting(iq_signal, center=True, detrend=True, clip_std=10.0, scale_to_rms=True):
    sig = np.asarray(iq_signal, dtype=np.complex128)
    if len(sig) == 0:
        return np.array([0 + 0j])
    I = np.real(sig).astype(np.float64)
    Q = np.imag(sig).astype(np.float64)
    if center:
        I -= np.mean(I)
        Q -= np.mean(Q)
    if detrend:
        try:
            I = signal.detrend(I)
            Q = signal.detrend(Q)
        except:
            pass
    # Optional IQR clipping
    if CLIP_OUTLIERS and clip_std > 0:
        for comp in [I, Q]:
            q1, q3 = np.percentile(comp, [25, 75])
            iqr = q3 - q1
            lower = q1 - 1.5 * iqr
            upper = q3 + 1.5 * iqr
            comp = np.clip(comp, lower, upper)
    if scale_to_rms:
        rms = np.sqrt(np.mean(I**2 + Q**2) + 1e-12)
        if rms > 0:
            I /= rms
            Q /= rms
    else:
        I += np.random.normal(0, 1e-6, len(I))
        Q += np.random.normal(0, 1e-6, len(Q))
    return I + 1j * Q

In [4]:
# Cell 4: Feature extraction (Full original + MPACT RC-burst feats)
def extract_comprehensive_iq_features(iq_signal, fs, metadata):
    eps = 1e-12
    features = {}
    if len(iq_signal) == 0:
        return {k: 0.0 for k in ['signal_length', 'sampling_rate']} # Handle zero-length
    # Cast to higher precision
    iq_signal = iq_signal.astype(np.complex128)
    # Normalize to unit RMS to prevent overflows and make features comparable
    if NORMALIZE_BEFORE_FEATURES:
        abs_sq = np.abs(iq_signal)**2
        mean_abs_sq = np.mean(abs_sq)
        rms = np.sqrt(mean_abs_sq + eps)
        if rms > 0:
            iq_signal /= rms
        features['normalized_rms'] = True
    else:
        features['normalized_rms'] = False
    I = np.real(iq_signal)
    Q = np.imag(iq_signal)
    magnitude = np.abs(iq_signal)
    phase = np.angle(iq_signal)
    phase_unwrapped = np.unwrap(phase)

    features['signal_length'] = int(len(iq_signal))
    features['sampling_rate'] = float(fs)

    # magnitude
    features['mag_mean'] = float(np.mean(magnitude))
    features['mag_std'] = float(np.std(magnitude))
    features['mag_var'] = float(np.var(magnitude))
    features['mag_rms'] = float(np.sqrt(np.mean(magnitude**2)))
    features['mag_max'] = float(np.max(magnitude) if magnitude.size>0 else 0.0)
    features['mag_min'] = float(np.min(magnitude) if magnitude.size>0 else 0.0)
    features['mag_range'] = features['mag_max'] - features['mag_min']
    features['mag_peak_to_avg'] = features['mag_max'] / (features['mag_mean'] + eps)
    features['mag_skewness'] = float(stats.skew(magnitude))
    features['mag_kurtosis'] = float(stats.kurtosis(magnitude))

    # I
    features['i_mean'] = float(np.mean(I))
    features['i_std'] = float(np.std(I))
    features['i_var'] = float(np.var(I))
    features['i_skewness'] = float(stats.skew(I))
    features['i_kurtosis'] = float(stats.kurtosis(I))
    features['i_rms'] = float(np.sqrt(np.mean(I**2)))

    # Q
    features['q_mean'] = float(np.mean(Q))
    features['q_std'] = float(np.std(Q))
    features['q_var'] = float(np.var(Q))
    features['q_skewness'] = float(stats.skew(Q))
    features['q_kurtosis'] = float(stats.kurtosis(Q))
    features['q_rms'] = float(np.sqrt(np.mean(Q**2)))

    # IQ balance
    features['iq_imbalance_dc'] = float(np.abs(np.mean(I) - np.mean(Q)))
    features['iq_imbalance_power_db'] = float(10 * np.log10((np.var(I) + eps) / (np.var(Q) + eps)))
    try:
        features['iq_correlation'] = float(np.corrcoef(I, Q)[0, 1])
    except Exception:
        features['iq_correlation'] = 0.0
    features['iq_power_ratio'] = float(np.var(I) / (np.var(Q) + eps))
    features['iq_amplitude_ratio'] = float(np.mean(np.abs(I)) / (np.mean(np.abs(Q)) + eps))

    # phase stats
    features['phase_mean'] = float(np.mean(phase))
    features['phase_std'] = float(np.std(phase))
    features['phase_var'] = float(np.var(phase))
    features['phase_range'] = float(np.max(phase) - np.min(phase))
    features['phase_skewness'] = float(stats.skew(phase))
    features['phase_kurtosis'] = float(stats.kurtosis(phase))

    # unwrapped
    if len(phase_unwrapped) >= 2:
        features['phase_unwrapped_slope'] = float(np.polyfit(np.arange(len(phase_unwrapped)), phase_unwrapped, 1)[0])
    else:
        features['phase_unwrapped_slope'] = 0.0
    features['phase_unwrapped_mean'] = float(np.mean(phase_unwrapped))
    features['phase_unwrapped_std'] = float(np.std(phase_unwrapped))
    features['phase_unwrapped_range'] = float(np.max(phase_unwrapped) - np.min(phase_unwrapped))

    # instantaneous freq
    inst_freq = np.diff(phase_unwrapped) * fs / (2 * np.pi) if len(phase_unwrapped) >= 2 else np.array([])
    if inst_freq.size > 0:
        features['inst_freq_mean'] = float(np.mean(inst_freq))
        features['inst_freq_std'] = float(np.std(inst_freq))
        features['inst_freq_var'] = float(np.var(inst_freq))
        features['inst_freq_range'] = float(np.max(inst_freq) - np.min(inst_freq))
        features['inst_freq_skewness'] = float(stats.skew(inst_freq))
        features['inst_freq_kurtosis'] = float(stats.kurtosis(inst_freq))
        freq_accel = np.diff(inst_freq) * fs / (2 * np.pi) if len(inst_freq) >= 2 else np.array([])
        if freq_accel.size > 0:
            features['freq_acceleration_mean'] = float(np.mean(freq_accel))
            features['freq_acceleration_std'] = float(np.std(freq_accel))
        else:
            features['freq_acceleration_mean'] = 0.0
            features['freq_acceleration_std'] = 0.0
    else:
        features.update({
            'inst_freq_mean': 0.0, 'inst_freq_std': 0.0, 'inst_freq_var': 0.0,
            'inst_freq_range': 0.0, 'inst_freq_skewness': 0.0, 'inst_freq_kurtosis': 0.0,
            'freq_acceleration_mean': 0.0, 'freq_acceleration_std': 0.0
        })

    # AM features
    features['am_depth'] = float((features['mag_max'] - features['mag_min']) / (features['mag_max'] + features['mag_min'] + eps))
    features['amplitude_variation'] = float(np.std(magnitude) / (np.mean(magnitude) + eps))

    # EVM
    try:
        norm = np.mean(np.abs(iq_signal))
        if norm <= eps:
            features['evm'] = 0.0
            features['evm_rms'] = 0.0
        else:
            normalized_signal = iq_signal / norm
            ideal_phase = np.round(np.angle(normalized_signal) / (np.pi/4)) * (np.pi/4)
            ideal_signal = np.exp(1j * ideal_phase)
            error_vector = normalized_signal - ideal_signal
            features['evm'] = float(np.mean(np.abs(error_vector)))
            features['evm_rms'] = float(np.sqrt(np.mean(np.abs(error_vector)**2)))
    except Exception:
        features['evm'] = 0.0
        features['evm_rms'] = 0.0

    # spectral features
    try:
        nperseg = min(4096, max(8, len(magnitude)//4))
        if nperseg > len(magnitude):
            nperseg = max(1, len(magnitude))
        f_mag, psd_mag = welch(magnitude, fs=fs, nperseg=nperseg)
        total_power = np.sum(psd_mag) + eps
        normalized_psd = psd_mag / total_power
        features['spectral_centroid_mag'] = float(np.sum(f_mag * normalized_psd))
        features['spectral_bandwidth_mag'] = float(np.sqrt(np.sum(((f_mag - features['spectral_centroid_mag']) ** 2) * normalized_psd)))
        cumsum_psd = np.cumsum(normalized_psd)
        rolloff_85_idx = np.where(cumsum_psd >= 0.85)[0]
        rolloff_95_idx = np.where(cumsum_psd >= 0.95)[0]
        features['spectral_rolloff_85'] = float(f_mag[rolloff_85_idx[0]] if len(rolloff_85_idx) > 0 else f_mag[-1])
        features['spectral_rolloff_95'] = float(f_mag[rolloff_95_idx[0]] if len(rolloff_95_idx) > 0 else f_mag[-1])
    except Exception:
        features.update({
            'spectral_centroid_mag': 0.0, 'spectral_bandwidth_mag': 0.0,
            'spectral_rolloff_85': 0.0, 'spectral_rolloff_95': 0.0
        })

    # circular stats
    try:
        circ_mean = np.angle(np.mean(np.exp(1j * phase)))
        circ_var = 1 - np.abs(np.mean(np.exp(1j * phase)))
        features['phase_circular_mean'] = float(circ_mean)
        features['phase_circular_variance'] = float(circ_var)
        features['phase_circular_std'] = float(np.sqrt(-2 * np.log(1 - circ_var + eps))) if (0 <= circ_var < 1) else 0.0
    except Exception:
        features['phase_circular_mean'] = 0.0
        features['phase_circular_variance'] = 0.0
        features['phase_circular_std'] = 0.0

    # NEW: MPACT RC-burst features (duty cycle, peak count for 0.25ms bursts)
    mag_thresh = np.mean(magnitude) + 2 * np.std(magnitude)
    burst_mask = magnitude > mag_thresh
    features['burst_duty_cycle'] = float(np.sum(burst_mask) / len(burst_mask))
    peaks, properties = find_peaks(magnitude, height=mag_thresh)
    features['num_peaks'] = float(len(peaks))
    if 'prominences' in properties and len(properties['prominences']) > 0:
        features['peak_prominence_mean'] = float(np.mean(properties['prominences']))
    else:
        features['peak_prominence_mean'] = 0.0

    # Force GC after feature extraction
    del I, Q, magnitude, phase, phase_unwrapped, inst_freq
    gc.collect()

    return features

In [5]:
# Cell 5: Plotting functions (Full impl with high-res histograms; centralized DL)
def create_constellation_diagram(iq_signal, basename, out_subdir='original_constellation_plots', center=False, detrend=False, clip_std=10.0, scale_to_rms=False, out_dir=OUTPUT_DIR, max_points=20000, dpi=DPI_SAVE, dl_ready=False):
    try:
        sig = iq_signal
        n = len(sig)
        if n > max_points:
            idx = np.linspace(0, n - 1, max_points, dtype=int)
            sig = sig[idx]
        sig_plot = prepare_iq_for_plotting(sig, center=center, detrend=detrend, clip_std=clip_std, scale_to_rms=scale_to_rms or dl_ready) # Force scale for DL
        I = np.real(sig_plot)
        Q = np.imag(sig_plot)
        # Dynamic limits for better visibility, unless DL-ready (fixed for consistency)
        if dl_ready:
            lim = 1.5 # Tight fixed lim post-normalization
        else:
            max_val = max(np.max(np.abs(I)), np.max(np.abs(Q))) if len(I) > 0 else 1.0
            lim = max_val * 1.1 + 1e-6 # Avoid zero lim
        fig = plt.figure(figsize=(6, 6) if not dl_ready else (DL_IMAGE_SIZE[0]/DPI_SAVE_DL, DL_IMAGE_SIZE[1]/DPI_SAVE_DL), dpi=dpi if not dl_ready else DPI_SAVE_DL)
        hb = plt.hexbin(I, Q, gridsize=120, mincnt=1, bins='log', cmap='viridis')
        if not dl_ready:
            cb = plt.colorbar(hb)
            cb.set_label('log(count)')
            plt.xlabel('In-phase (I)')
            plt.ylabel('Quadrature (Q)')
            plt.title(f'Constellation Diagram - {basename}')
            plt.grid(True, alpha=0.3)
        else:
            plt.axis('off')
            plt.xlim(-lim, lim)
            plt.ylim(-lim, lim)
            plt.gca().set_aspect('equal', adjustable='box')
        subdir = out_subdir if not dl_ready else f'dl_ready_{out_subdir.split("_")[0]}_constellation_plots'
        outpath = os.path.join(out_dir, subdir, f'{basename}_constellation.png')
        plt.tight_layout(pad=0)
        plt.savefig(outpath, dpi=dpi if not dl_ready else DPI_SAVE_DL, bbox_inches='tight', pad_inches=0)
        plt.close(fig) # Explicit close
        plt.close('all') # Close any lingering figures
        gc.collect() # GC after plotting
        return True, outpath
    except Exception as e:
        plt.close('all')
        gc.collect()
        return False, None

# NEW: Centralized histogram plotter for high-res I/Q
def create_distribution_plot(component, iq_signal, basename, out_dir=OUTPUT_DIR, dpi=DPI_SAVE, dl_ready=False, center=True, detrend=True, clip_std=10.0, scale_to_rms=True):
    try:
        sig_plot = prepare_iq_for_plotting(iq_signal, center=center, detrend=detrend, clip_std=clip_std, scale_to_rms=scale_to_rms or dl_ready)
        if component == 'I':
            vals = np.real(sig_plot)
            color = 'blue'
        else:
            vals = np.imag(sig_plot)
            color = 'red'
        bins = HIST_BINS  # Use full bins for both
        fig = plt.figure(figsize=(6,4) if not dl_ready else (DL_IMAGE_SIZE[0]/DPI_SAVE_DL, DL_IMAGE_SIZE[1]/DPI_SAVE_DL), dpi=dpi if not dl_ready else DPI_SAVE_DL)
        plt.hist(vals, bins=bins, alpha=0.7, color=color, density=True)
        if not dl_ready:
            plt.xlabel(f'{component} Value')
            plt.ylabel('Density')
            plt.title(f'{component} Component Distribution - {basename}')
            plt.grid(True, alpha=0.3)
        else:
            plt.axis('off')
        subdir = f'{component.lower()}_distribution_plots' if not dl_ready else f'dl_ready_{component.lower()}_distribution_plots'
        outpath = os.path.join(out_dir, subdir, f'{basename}_{component.lower()}_distribution.png')
        plt.tight_layout(pad=0)
        plt.savefig(outpath, dpi=dpi if not dl_ready else DPI_SAVE_DL, bbox_inches='tight', pad_inches=0)
        plt.close(fig)
        plt.close('all')
        gc.collect()
        return True, outpath
    except Exception as e:
        plt.close('all')
        gc.collect()
        return False, None

def create_i_distribution_plot(iq_signal, basename, out_dir=OUTPUT_DIR, dpi=DPI_SAVE, dl_ready=False, center=True, detrend=True, clip_std=10.0, scale_to_rms=True):
    return create_distribution_plot('I', iq_signal, basename, out_dir, dpi, dl_ready, center, detrend, clip_std, scale_to_rms)

def create_q_distribution_plot(iq_signal, basename, out_dir=OUTPUT_DIR, dpi=DPI_SAVE, dl_ready=False, center=True, detrend=True, clip_std=10.0, scale_to_rms=True):
    return create_distribution_plot('Q', iq_signal, basename, out_dir, dpi, dl_ready, center, detrend, clip_std, scale_to_rms)

# Corrected version of the create_magnitude_phase_plot function
# Changes:
# - Switched from scatter to hist2d (2D histogram) to aggregate points and reduce blurriness from overlapping scatters, especially in small DL-ready images (224x224).
# - Used cmap='gray' to keep it "normal" (not colorful), focusing on density without unnecessary colors.
# - For non-DL-ready plots, added a colorbar labeled 'Density' to show the histogram density.
# - For DL-ready, no colorbar or labels, just the grayscale histogram image.
# - Reduced bins to 30 for DL-ready to avoid too fine-grained bins that could look noisy/blurry in small sizes; kept 50 for regular plots.
# - This should make DL-ready images sharper and less blurry, as hist2d creates clear density maps without point overlaps.

def create_magnitude_phase_plot(iq_signal, basename, out_dir=OUTPUT_DIR, dpi=DPI_SAVE, dl_ready=False, center=True, detrend=True, clip_std=10.0, scale_to_rms=True, max_points=20000):
    try:
        sig = iq_signal
        n = len(sig)
        if n > max_points:
            idx = np.linspace(0, n - 1, max_points, dtype=int)
            sig = sig[idx]
        sig_plot = prepare_iq_for_plotting(sig, center=center, detrend=detrend, clip_std=clip_std, scale_to_rms=scale_to_rms or dl_ready)
        magnitude = np.abs(sig_plot)
        phase = np.angle(sig_plot)
        fig = plt.figure(figsize=(6,4) if not dl_ready else (DL_IMAGE_SIZE[0]/DPI_SAVE_DL, DL_IMAGE_SIZE[1]/DPI_SAVE_DL), dpi=dpi if not dl_ready else DPI_SAVE_DL)
        bins = 50 if not dl_ready else 30  # Fewer bins for DL-ready to reduce noise/blur in small images
        plt.hist2d(phase, magnitude, bins=bins, cmap='gray', density=True)
        if not dl_ready:
            cb = plt.colorbar()
            cb.set_label('Density')
            plt.xlabel('Phase (rad)')
            plt.ylabel('Magnitude')
            plt.title(f'Magnitude vs Phase - {basename}')
            plt.grid(True, alpha=0.3)
        else:
            plt.axis('off')
        subdir = 'magnitude_phase_plots' if not dl_ready else 'dl_ready_magnitude_phase_plots'
        outpath = os.path.join(out_dir, subdir, f'{basename}_magnitude_phase.png')
        plt.tight_layout(pad=0)
        plt.savefig(outpath, dpi=dpi if not dl_ready else DPI_SAVE_DL, bbox_inches='tight', pad_inches=0)
        plt.close(fig)
        plt.close('all')
        gc.collect()
        del magnitude, phase  # Clean up
        return True, outpath
    except Exception as e:
        plt.close('all')
        gc.collect()
        return False, None
    
    
def create_comprehensive_spectrograms(iq_signal, fs, basename, out_dir=OUTPUT_DIR, dpi=DPI_SAVE):
    try:
        I = np.real(iq_signal)
        Q = np.imag(iq_signal)
        magnitude = np.abs(iq_signal)
        phase = np.angle(iq_signal)
        phase_unwrapped = np.unwrap(phase)
        inst_freq = np.diff(phase_unwrapped) * fs / (2 * np.pi) if len(phase_unwrapped) >= 2 else np.array([])
        signal_length = len(iq_signal)
        nperseg = min(4096, max(256, signal_length // 8)) if signal_length > 0 else 256
        if nperseg > signal_length:
            nperseg = max(1, signal_length)
        noverlap = max(0, nperseg // 2)
        f_mag, t_mag, Zxx_mag = signal.spectrogram(iq_signal, fs=fs, window='hann', nperseg=nperseg, noverlap=noverlap, return_onesided=False, mode='complex')
        magnitude_spectrogram = np.abs(Zxx_mag)
        magnitude_db = 20 * np.log10(magnitude_spectrogram + 1e-12)
        mag_flat = magnitude_db.flatten()
        vmin = np.percentile(mag_flat, 2)
        vmax = np.percentile(mag_flat, 98)
        # Combined figure (annotated)
        fig_combined = plt.figure(figsize=(14,10), dpi=dpi)
        plt.subplot(2,2,1)
        f_mhz = fftshift(f_mag) / 1e6
        t_ms = t_mag * 1000
        mag_shift = fftshift(magnitude_db, axes=0)
        plt.pcolormesh(t_ms, f_mhz, mag_shift, shading='gouraud', vmin=vmin, vmax=vmax, cmap='viridis')
        plt.colorbar(label='Magnitude (dB)')
        plt.xlabel('Time (ms)')
        plt.ylabel('Frequency (MHz)')
        plt.title(f'Magnitude Spectrogram - {basename}')
        plt.subplot(2,2,2)
        phase_spec = np.angle(Zxx_mag)
        phase_shift = fftshift(np.unwrap(phase_spec, axis=1), axes=0)
        plt.pcolormesh(t_ms, f_mhz, phase_shift, shading='gouraud', cmap='hsv')
        plt.colorbar(label='Phase (rad)')
        plt.xlabel('Time (ms)')
        plt.ylabel('Frequency (MHz)')
        plt.title(f'Phase Spectrogram - {basename}')
        plt.subplot(2,2,3)
        if inst_freq.size > 0:
            # NEW: Subsample inst_freq for plotting to prevent distortion/OOM
            max_inst_points = 10000
            if len(inst_freq) > max_inst_points:
                step = len(inst_freq) // max_inst_points
                inst_idx = slice(0, -1, step) # Avoid last incomplete
                t_inst = np.arange(len(inst_freq))[inst_idx] / fs * 1000
                inst_freq_plot = inst_freq[inst_idx]
            else:
                t_inst = np.arange(len(inst_freq)) / fs * 1000
                inst_freq_plot = inst_freq
            plt.plot(t_inst, inst_freq_plot / 1e6, linewidth=0.6)
        plt.xlabel('Time (ms)')
        plt.ylabel('Inst. Freq (MHz)')
        plt.title(f'Instantaneous Frequency - {basename}')
        plt.grid(True, alpha=0.3)
        plt.subplot(2,2,4)
        max_pts = min(10000, signal_length)
        t_samples = np.arange(max_pts) / fs * 1e6 if fs > 0 else np.arange(max_pts)
        plt.plot(t_samples, I[:max_pts], linewidth=0.5, alpha=0.8, label='I')
        plt.plot(t_samples, Q[:max_pts], linewidth=0.5, alpha=0.8, label='Q')
        plt.xlabel('Time (µs)')
        plt.ylabel('Amplitude')
        plt.legend()
        plt.title(f'I/Q Time Series - {basename}')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        combined_path = os.path.join(out_dir, 'combined_visualizations', f'{basename}_comprehensive.png')
        plt.savefig(combined_path, dpi=dpi, bbox_inches='tight')
        plt.close(fig_combined)
        # magnitude (annotated)
        fig_mag = plt.figure(figsize=(10,6), dpi=dpi)
        plt.pcolormesh(t_ms, f_mhz, mag_shift, shading='gouraud', vmin=vmin, vmax=vmax, cmap='viridis')
        plt.colorbar(label='Magnitude (dB)')
        plt.xlabel('Time (ms)')
        plt.ylabel('Frequency (MHz)')
        plt.title(f'Magnitude Spectrogram - {basename}')
        plt.tight_layout()
        mag_path = os.path.join(out_dir, 'magnitude_spectrogram_plots', f'{basename}_magnitude.png')
        plt.savefig(mag_path, dpi=dpi, bbox_inches='tight')
        plt.close(fig_mag)
        # DL-ready magnitude spectrogram: no annotations
        fig_dl_mag = plt.figure(figsize=(DL_IMAGE_SIZE[0]/DPI_SAVE_DL, DL_IMAGE_SIZE[1]/DPI_SAVE_DL), dpi=DPI_SAVE_DL)
        plt.pcolormesh(t_ms, f_mhz, mag_shift, shading='gouraud', vmin=vmin, vmax=vmax, cmap='viridis')
        plt.axis('off')
        plt.tight_layout(pad=0)
        dl_mag_path = os.path.join(out_dir, 'dl_ready_magnitude_spectrograms', f'{basename}_magnitude.png')
        plt.savefig(dl_mag_path, dpi=DPI_SAVE_DL, bbox_inches='tight', pad_inches=0)
        plt.close(fig_dl_mag)
        # DL-ready phase spectrogram
        fig_dl_phase = plt.figure(figsize=(DL_IMAGE_SIZE[0]/DPI_SAVE_DL, DL_IMAGE_SIZE[1]/DPI_SAVE_DL), dpi=DPI_SAVE_DL)
        plt.pcolormesh(t_ms, f_mhz, phase_shift, shading='gouraud', cmap='hsv')
        plt.axis('off')
        plt.tight_layout(pad=0)
        dl_phase_path = os.path.join(out_dir, 'dl_ready_phase_spectrograms', f'{basename}_phase.png')
        plt.savefig(dl_phase_path, dpi=DPI_SAVE_DL, bbox_inches='tight', pad_inches=0)
        plt.close(fig_dl_phase)
        # phase
        fig_phase = plt.figure(figsize=(10,6), dpi=dpi)
        plt.pcolormesh(t_ms, f_mhz, phase_shift, shading='gouraud', cmap='hsv')
        plt.colorbar(label='Phase (rad)')
        plt.xlabel('Time (ms)')
        plt.ylabel('Frequency (MHz)')
        plt.title(f'Phase Spectrogram - {basename}')
        plt.tight_layout()
        phase_path = os.path.join(out_dir, 'phase_spectrogram_plots', f'{basename}_phase.png')
        plt.savefig(phase_path, dpi=dpi, bbox_inches='tight')
        plt.close(fig_phase)
        # inst freq
        fig_inst = plt.figure(figsize=(10,4), dpi=dpi)
        if inst_freq.size > 0:
            # Reuse subsampled from above
            plt.plot(t_inst, inst_freq_plot / 1e6, linewidth=0.6)
        plt.xlabel('Time (ms)')
        plt.ylabel('Inst. Freq (MHz)')
        plt.title(f'Instantaneous Frequency - {basename}')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        inst_path = os.path.join(out_dir, 'instantaneous_frequency_plots', f'{basename}_instfreq.png')
        plt.savefig(inst_path, dpi=dpi, bbox_inches='tight')
        plt.close(fig_inst)
        # DL-ready inst freq
        fig_dl_inst = plt.figure(figsize=(DL_IMAGE_SIZE[0]/DPI_SAVE_DL, DL_IMAGE_SIZE[1]/DPI_SAVE_DL), dpi=DPI_SAVE_DL)
        if inst_freq.size > 0:
            plt.plot(t_inst, inst_freq_plot / 1e6, linewidth=0.6)
        plt.axis('off')
        plt.tight_layout(pad=0)
        dl_inst_path = os.path.join(out_dir, 'dl_ready_instantaneous_frequency_plots', f'{basename}_instfreq.png')
        plt.savefig(dl_inst_path, dpi=DPI_SAVE_DL, bbox_inches='tight', pad_inches=0)
        plt.close(fig_dl_inst)
        # iq timeseries
        fig_iq = plt.figure(figsize=(10,4), dpi=dpi)
        plt.plot(t_samples, I[:max_pts], linewidth=0.6, label='I')
        plt.plot(t_samples, Q[:max_pts], linewidth=0.6, label='Q')
        plt.xlabel('Time (µs)')
        plt.ylabel('Amplitude')
        plt.legend()
        plt.title(f'I/Q Time Series - {basename}')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        iq_path = os.path.join(out_dir, 'iq_time_series_plots', f'{basename}_iq.png')
        plt.savefig(iq_path, dpi=dpi, bbox_inches='tight')
        plt.close(fig_iq)
        # DL-ready iq timeseries
        fig_dl_iq = plt.figure(figsize=(DL_IMAGE_SIZE[0]/DPI_SAVE_DL, DL_IMAGE_SIZE[1]/DPI_SAVE_DL), dpi=DPI_SAVE_DL)
        max_pts = min(10000, signal_length)
        t_samples = np.arange(max_pts) / fs * 1e6 if fs > 0 else np.arange(max_pts)
        plt.plot(t_samples, I[:max_pts], linewidth=0.6)
        plt.plot(t_samples, Q[:max_pts], linewidth=0.6)
        plt.axis('off')
        plt.tight_layout(pad=0)
        dl_iq_path = os.path.join(out_dir, 'dl_ready_iq_time_series_plots', f'{basename}_iq.png')
        plt.savefig(dl_iq_path, dpi=DPI_SAVE_DL, bbox_inches='tight', pad_inches=0)
        plt.close(fig_dl_iq)
        # Clean up large arrays
        del Zxx_mag, magnitude_spectrogram, magnitude_db, mag_shift, phase_shift, f_mag, t_mag
        gc.collect()
        plt.close('all')
        return True, {'combined': combined_path, 'magnitude': mag_path, 'phase': phase_path, 'inst_freq': inst_path, 'iq_plot': iq_path, 'dl_magnitude': dl_mag_path, 'dl_phase': dl_phase_path, 'dl_inst_freq': dl_inst_path, 'dl_iq': dl_iq_path}
    except Exception as e:
        plt.close('all')
        gc.collect()
        return False, None

In [6]:
# Cell 6: Worker function (Updated calls for high-res plots)
def process_file(file_path, out_dir=OUTPUT_DIR, test_mode=TEST_MODE, counter=None, lock=None):
    try:
        iq_signal, metadata = load_iq_signal(file_path, test_mode=test_mode, max_signal_len=MAX_SIGNAL_LEN)
        if iq_signal is None:
            return None
        feats = extract_comprehensive_iq_features(iq_signal, metadata['fs'], metadata)
        feats['device'] = metadata.get('device', '')
        feats['device_class'] = metadata.get('device_class', '')  # Multi-class from device name
        feats['filename'] = metadata.get('filename', os.path.basename(file_path))
        # save raw iq
        basename = os.path.splitext(feats['filename'])[0]
        raw_path = os.path.join(out_dir, 'raw_iq_signals', f'{basename}_iq.npy')
        try:
            np.save(raw_path, iq_signal)
            feats['raw_saved'] = raw_path
        except Exception as e:
            feats['raw_saved'] = ''
        # create plots (separate saving handled inside)
        create_comprehensive_spectrograms(iq_signal, metadata['fs'], basename, out_dir=out_dir)
        # Original constellation: clipped but no center/detrend/scale
        create_constellation_diagram(iq_signal, basename, out_subdir='original_constellation_plots', center=False, detrend=False, clip_std=FIXED_CLIP_STD, scale_to_rms=False, out_dir=out_dir)
        # Processed/zoomed constellation: centered, clipped, scaled
        create_constellation_diagram(iq_signal, basename, out_subdir='processed_constellation_plots', center=True, detrend=True, clip_std=FIXED_CLIP_STD, scale_to_rms=True, out_dir=out_dir)
        # DL-ready versions (force scaling for consistency)
        create_constellation_diagram(iq_signal, basename, out_subdir='original_constellation_plots', center=False, detrend=False, clip_std=FIXED_CLIP_STD, scale_to_rms=True, out_dir=out_dir, dl_ready=True)
        create_constellation_diagram(iq_signal, basename, out_subdir='processed_constellation_plots', center=True, detrend=True, clip_std=FIXED_CLIP_STD, scale_to_rms=True, out_dir=out_dir, dl_ready=True)
        # I distribution (processed, high-res)
        create_i_distribution_plot(iq_signal, basename, out_dir=out_dir, center=True, detrend=True, clip_std=FIXED_CLIP_STD, scale_to_rms=True)
        # DL-ready I distribution (high-res subsampled)
        create_i_distribution_plot(iq_signal, basename, out_dir=out_dir, dl_ready=True, center=True, detrend=True, clip_std=FIXED_CLIP_STD, scale_to_rms=True)
        # Q distribution (processed, high-res)
        create_q_distribution_plot(iq_signal, basename, out_dir=out_dir, center=True, detrend=True, clip_std=FIXED_CLIP_STD, scale_to_rms=True)
        # DL-ready Q distribution (high-res subsampled)
        create_q_distribution_plot(iq_signal, basename, out_dir=out_dir, dl_ready=True, center=True, detrend=True, clip_std=FIXED_CLIP_STD, scale_to_rms=True)
        # Magnitude vs Phase (processed)
        create_magnitude_phase_plot(iq_signal, basename, out_dir=out_dir, center=True, detrend=True, clip_std=FIXED_CLIP_STD, scale_to_rms=True)
        # DL-ready Magnitude vs Phase
        create_magnitude_phase_plot(iq_signal, basename, out_dir=out_dir, dl_ready=True, center=True, detrend=True, clip_std=FIXED_CLIP_STD, scale_to_rms=True)
        # Final cleanup before return
        del iq_signal
        gc.collect()
        return feats
    except Exception as e:
        logging.error(f"Process error {file_path}: {e}")
        plt.close('all')
        gc.collect()
        return None
    finally:
        if counter is not None and lock is not None:
            with lock:
                counter.value += 1

In [ ]:
# Cell 7: Collect files and run parallel processing
all_files = []
device_counts = {}
for root, _, files in os.walk(DATASET_DIR):
    mats = [f for f in files if f.endswith('.mat')]
    if mats:
        device = os.path.basename(root)
        take = mats[:MAX_FILES_PER_DEVICE] if MAX_FILES_PER_DEVICE else mats
        device_counts[device] = len(take)
        for m in take:
            all_files.append(os.path.join(root, m))
print(f'Found {len(all_files)} .mat files across {len(device_counts)} devices')
if len(all_files) == 0:
    raise RuntimeError('No .mat files found. Set DATASET_DIR correctly and re-run this cell.')
if TEST_MODE:
    print('TEST_MODE=true — processing small subset per config')

print('Starting parallel processing...')
manager = multiprocessing.Manager()
counter = manager.Value('i', 0)
lock = manager.Lock()
progress = IntProgress(min=0, max=len(all_files), description='Processing:')
display(progress)
def update_slider():
    while counter.value < len(all_files):
        progress.value = counter.value
        time.sleep(0.1)
    progress.value = len(all_files)
t = Thread(target=update_slider)
t.start()
# Use loky backend for process isolation, but limit cores
results = Parallel(n_jobs=NUM_CORES, backend='loky', verbose=0, max_nbytes=1e6)(
    delayed(process_file)(f, out_dir=OUTPUT_DIR, test_mode=TEST_MODE, counter=counter, lock=lock) for f in all_files
)
t.join()
results = [r for r in results if r is not None]
print('Parallel processing finished — successful:', len(results))
gc.collect() # Final GC

In [ ]:
# Cell 8: Clean numeric data, save CSV, and ML (Multi-class + importance)
def clean_numeric_array(X, clip_value=1e12):
    X = np.asarray(X, dtype=np.float64)
    X[np.isinf(X)] = np.nan
    col_medians = np.nanmedian(X, axis=0)
    col_medians = np.where(np.isnan(col_medians), 0.0, col_medians)
    inds = np.where(np.isnan(X))
    if inds[0].size > 0:
        X[inds] = np.take(col_medians, inds[1])
    X = np.clip(X, -clip_value, clip_value)
    return X

if not results:
    print('No results to analyze')
else:
    features_df = pd.DataFrame(results)
    features_csv = os.path.join(OUTPUT_DIR, 'extracted_features', 'mpact_iq_features.csv')
    features_df.to_csv(features_csv, index=False)
    print('Saved MPACT features to', features_csv)
    numeric_df = features_df.select_dtypes(include=[np.number]).copy()
    print('Numeric features shape:', numeric_df.shape)
    X_raw = numeric_df.values
    X_clean = clean_numeric_array(X_raw, clip_value=1e12)
    X_clean_df = pd.DataFrame(X_clean, columns=numeric_df.columns)
    print(features_df.shape)
    

In [ ]:
# Cell 9: Output folder summary
summary = {}
for sd in subdirs:
    p = os.path.join(OUTPUT_DIR, sd)
    if os.path.exists(p):
        summary[sd] = len([f for f in os.listdir(p) if os.path.isfile(os.path.join(p, f))])
    else:
        summary[sd] = 0
print('Output summary:')
for k, v in summary.items():
    print(f' {k}: {v} files')